# Needleman-Wunsch Algorithm Theory

## Introduction

The Needleman-Wunsch algorithm is a dynamic programming algorithm used for pairwise global sequence alignment. It is commonly employed in bioinformatics to compare biological sequences, such as DNA, RNA, or protein sequences. The algorithm aims to find the optimal alignment of two sequences by maximizing a scoring function that considers matches, mismatches, and gap penalties.

## Scoring Function

Let's define the scoring function used in the algorithm:

- **Match Score ($S_{match}$):** The score assigned for a match between corresponding characters in the two sequences.
- **Mismatch Penalty ($S_{mismatch}$):** The penalty for a mismatch between corresponding characters.
- **Gap Penalty ($S_{gap}$):** The penalty for introducing a gap in one of the sequences.

The scoring function for aligning characters $(a_i, b_j)$ is given by:

$$S(a_i, b_j) = \begin{cases} 
S_{\text{match}} & \text{if } a_i = b_j \\
S_{\text{mismatch}} & \text{if } a_i \neq b_j 
\end{cases}$$

## Scoring Matrix

In the context of the Needleman-Wunsch algorithm, a scoring matrix, denoted as $M$, plays a crucial role in efficiently computing the optimal alignment between two sequences ($\text{seq}_1$ and $\text{seq}_2$). This matrix is designed to store the optimal scores for aligning subsequences up to various positions in both sequences.

Each cell in the matrix, represented as $M[i, j]$, signifies the optimal score for aligning the subsequences $\text{seq}_1[1:i]$ and $\text{seq}_2[1:j]$. The matrix is constructed using a recurrence relation that considers three possible scenarios:

1. **Diagonal (Match/Mismatch):**
   $M[i, j] = M[i-1, j-1] + S(a_i, b_j)$.
   This accounts for aligning characters $a_i$ and $b_j$ at positions $i$ and $j$, respectively, considering a match or mismatch according to the scoring function $S$.

2. **Up (Gap in Sequence1 ($\text{seq}_1$)):**
   $M[i, j] = M[i-1, j] + S_{\text{gap}}$.
   This represents the case where a gap is introduced in sequence $\text{seq}_1$ at position $i$.

3. **Left (Gap in Sequence2 ($\text{seq}_2$)):**
   $M[i, j] = M[i, j-1] + S_{\text{gap}}$.
   This corresponds to introducing a gap in sequence $\text{seq}_2$ at position $j$.

By iteratively applying these recurrence relations, the scoring matrix $M$ is populated, and the optimal alignment scores for various subsequences are computed, forming the basis for determining the overall optimal alignment between $\text{seq}_1$ and $\text{seq}_2$.

## Traceback

Once the matrix is filled, the optimal alignment is found by performing a traceback starting from the bottom-right cell of the matrix. The traceback follows the path of maximum scores.

## Example Usage

Let's demonstrate the implementation of the Needleman-Wunsch algorithm using an example with two DNA sequences. The DNA sequences are read from the protein_sequences.fasta file that is located in the repository.

In [3]:
def read_fasta(file_path):
    sequences = {}
    current_sequence = None

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                current_sequence = line[1:]
                sequences[current_sequence] = ''
            else:
                sequences[current_sequence] += line

    return sequences

def initialize_matrix(rows, cols):
    return [[0] * cols for _ in range(rows)]

def fill_scoring_matrix(seq1, seq2, gap_penalty, match_score, mismatch_penalty):
    len_seq1 = len(seq1)
    len_seq2 = len(seq2)

    # Initialize the scoring matrix
    score_matrix = initialize_matrix(len_seq1 + 1, len_seq2 + 1)

    # Initialize the first row and column
    for i in range(len_seq1 + 1):
        score_matrix[i][0] = i * gap_penalty

    for j in range(len_seq2 + 1):
        score_matrix[0][j] = j * gap_penalty

    # Fill in the scoring matrix
    for i in range(1, len_seq1 + 1):
        for j in range(1, len_seq2 + 1):
            match = score_matrix[i - 1][j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_penalty)
            delete = score_matrix[i - 1][j] + gap_penalty
            insert = score_matrix[i][j - 1] + gap_penalty

            score_matrix[i][j] = max(match, delete, insert)

    return score_matrix

def traceback(score_matrix, seq1, seq2, match_score=1, mismatch_penalty=-1, gap_penalty=-2):
    aligned_seq1 = ''
    aligned_seq2 = ''
    i, j = len(seq1), len(seq2)

    while i > 0 or j > 0:
        current_score = score_matrix[i][j]
        if i > 0 and j > 0:
            diagonal_score = score_matrix[i - 1][j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_penalty)
        else:
            diagonal_score = float('-inf')
        up_score = score_matrix[i - 1][j] + gap_penalty if i > 0 else float('-inf')
        left_score = score_matrix[i][j - 1] + gap_penalty if j > 0 else float('-inf')

        if current_score == diagonal_score:
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            i -= 1
            j -= 1
        elif current_score == up_score:
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = '-' + aligned_seq2
            i -= 1
        else:
            aligned_seq1 = '-' + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            j -= 1

    return aligned_seq1, aligned_seq2

def needleman_wunsch(seq1, seq2, gap_penalty=-2, match_score=1, mismatch_penalty=-1, verbose=False):
    # Fill the scoring matrix
    score_matrix = fill_scoring_matrix(seq1, seq2, gap_penalty, match_score, mismatch_penalty)
        
    if verbose == True:
        #Print the scoring matrix
        print("Scoring Matrix")
        for row in score_matrix:
            print(row)

     # Perform traceback
    aligned_seq1, aligned_seq2 = traceback(score_matrix, seq1, seq2, match_score, mismatch_penalty, gap_penalty)
    
    return aligned_seq1, aligned_seq2

if __name__ == "__main__":
    # Example usage
    fasta_file = "DNA_sequences.fasta"
    sequences = read_fasta(fasta_file)

    seq1_name, seq2_name = list(sequences.keys())[:2]
    seq1 = sequences[seq1_name]
    seq2 = sequences[seq2_name]

    aligned_seq1, aligned_seq2 = needleman_wunsch(seq1, seq2, verbose=True)

    print(f"Alignment for {seq1_name} and {seq2_name}:")
    print(aligned_seq1)
    print(aligned_seq2)


Scoring Matrix
[0, -2, -4, -6, -8, -10]
[-2, 1, -1, -3, -5, -7]
[-4, -1, 0, 0, -2, -4]
[-6, -3, 0, -1, 1, -1]
[-8, -5, -2, -1, -1, 2]
[-10, -7, -4, -3, -2, 0]
Alignment for Sequence1 and Sequence2:
A-CGAA
AGCG-A
